In [1]:
import polars as pl
import numpy as np
import holoviews as hv
from time import time

from cow_amm_trades import compute_cow_amm_trades
from balancer_pool_states import compute_balancer_pool_states
from coingecko import get_historical_token_price
from dataframes import (
    states_to_df,
    prices_to_df,
    combine_with_prices,
    plot_profit_vs_holding,
    compute_profit_vs_holding_apy,
)
from constants import (
    TOKEN1,
    TOKEN2,
    TOKEN1_COINGECKO,
    TOKEN2_COINGECKO,
)

[{'block': 32236569, 'WETH': 227250112810783827603, 'GNO': 1475000268143578981182997, 'time': 1708222403}, {'block': 32236874, 'WETH': 1475000266197371024185094, 'time': 1706797320}, {'block': 32236880, 'WETH': 1475000265070088030252733, 'time': 1706797350}, {'block': 32237338, 'WETH': 1475000264161438119193873, 'time': 1706799695}, {'block': 32237563, 'WETH': 1475000263282596669229866, 'time': 1706800905}, {'block': 32238449, 'WETH': 1475000241975531643752403, 'time': 1706805620}, {'block': 32238682, 'WETH': 1475000229177024285023265, 'time': 1706806865}, {'block': 32238844, 'WETH': 1475000187470817551227608, 'time': 1706807725}, {'block': 32238845, 'WETH': 1475000185494589702056754, 'time': 1706807730}, {'block': 32238986, 'WETH': 1475000162458310373496957, 'time': 1706808460}, {'block': 32240544, 'WETH': 1475000152939097315989676, 'time': 1706816645}, {'block': 32240606, 'WETH': 1475000151186960738577200, 'time': 1706816980}, {'block': 32250194, 'WETH': 1475000148445596561106176, 't

In [2]:
cow_amm_states = compute_cow_amm_trades()
balancer_pool_states = compute_balancer_pool_states()

In [3]:
df_cow = states_to_df(cow_amm_states)
df_balancer = states_to_df(balancer_pool_states)

KeyError: 'GNO'

In [6]:
token1_prices = get_historical_token_price(TOKEN1_COINGECKO)
token2_prices = get_historical_token_price(TOKEN2_COINGECKO)
token1_price_in_eth = get_historical_token_price(TOKEN1_COINGECKO, currency="eth")
token2_price_in_eth = get_historical_token_price(TOKEN2_COINGECKO, currency="eth")

df_token1_prices = prices_to_df(token1_prices)
df_token2_prices = prices_to_df(token2_prices)
df_token1_prices_in_eth = prices_to_df(token1_price_in_eth)
df_token2_prices_in_eth = prices_to_df(token2_price_in_eth)

In [7]:
from constants import START_TIME
from constants import START_TIME, PROTOCOL_FEE_WEI

time_now = time()
df_cow_filtered = df_cow.filter(pl.col("time") >= START_TIME)
#df_balancer_filtered = df_balancer.filter(pl.col("time") >= START_TIME)

In [8]:
df_cow_aggregate = combine_with_prices(df_cow_filtered, df_token1_prices, df_token2_prices)
#df_balancer_aggregate = combine_with_prices(df_balancer_filtered, df_weth_prices, df_cow_prices)

In [9]:
profit_vs_holding_cow = compute_profit_vs_holding_apy(df_cow_aggregate)
print(f"APY of CoW AMM: {profit_vs_holding_cow}")

protocol_fee_correction = (
    1
    + PROTOCOL_FEE_WEI
    / 10**18
    * df_cow_aggregate["price_"+TOKEN1][-1]
    / df_cow_aggregate["total_value"][-1]
)
profit_vs_holding_cow = compute_profit_vs_holding_apy(
    df_cow_aggregate, protocol_fee_correction
)
print(f"APY of CoW AMM (without paying protocol fees): {profit_vs_holding_cow}")
"""
profit_vs_holding_balancer = compute_profit_vs_holding_apy(df_balancer_aggregate)
print(f"APY of Balancer AMM: {profit_vs_holding_balancer}")"""

APY of CoW AMM: 1.0486937029879992
APY of CoW AMM (without paying protocol fees): 1.0318950414045288e+51


'\nprofit_vs_holding_balancer = compute_profit_vs_holding_apy(df_balancer_aggregate)\nprint(f"APY of Balancer AMM: {profit_vs_holding_balancer}")'

In [ ]:
trade_fraction = len(df_cow_filtered) / len(df_balancer_filtered)
volume_fraction = (
    df_cow_aggregate.with_columns(
        pl.col("value_weth").diff().abs().sum().alias("volume")
    )["volume"][0]
    / df_balancer_aggregate.with_columns(
        pl.col("value_weth").diff().abs().sum().alias("volume")
    )["volume"][0]
)
tvl_fraction = (
    df_cow_aggregate["total_value"][-1] / df_balancer_aggregate["total_value"][-1]
)
print(f"number of trades of CoW AMM vs balancer {trade_fraction * 100:2.1f}%")
print(f"volume of CoW AMM vs balancer {volume_fraction * 100:2.1f}%")
print(f"TVL of CoW AMM vs balancer {tvl_fraction * 100:2.1f}%")

number of trades of CoW AMM vs balancer 35.4%
volume of CoW AMM vs balancer 54.8%
TVL of CoW AMM vs balancer 64.2%


In [11]:
plt1 = plot_profit_vs_holding(df_cow_aggregate).relabel("cow amm")
plt2 = plot_profit_vs_holding(df_balancer_aggregate).relabel("balancer")
(plt1 * plt2).opts(legend_position="top_left", title="profid vs holding comparison")

SyntaxError: invalid syntax (2108571885.py, line 3)

In [10]:
plt1 = (
    df_token2_prices_in_eth.filter(pl.col("time") >= START_TIME)
    .with_columns(pl.from_epoch("time", time_unit="s"))
    .sort("time")
    .plot(x="time", y="price", label="price of COW in ETH")
)
plt2 = (
    df_token2_prices_in_eth.filter(pl.col("time") >= START_TIME)
    .with_columns(pl.from_epoch("time", time_unit="s"))
    .sort("time")
    .with_columns(
        pl.col("price")
        .rolling_var(by="time", window_size="12h")
        .alias("price_variance")
    )
    .plot(x="time", y="price_variance", label="price variance of COW in ETH")
)
plt1 + plt2

:Layout
   .Curve.Price_of_COW_in_ETH          :Curve   [time]   (price)
   .Curve.Price_variance_of_COW_in_ETH :Curve   [time]   (price_variance)